# Chapter 8: Integration

Robert Johansson

Source code listings for [Numerical Python - A Practical Techniques Approach for Industry](http://www.apress.com/9781484205549) (ISBN 978-1-484205-54-9).

The source code listings can be downloaded from http://www.apress.com/9781484205549

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.sans-serif'] = 'stix'

In [ ]:
import numpy as np

In [ ]:
from scipy import integrate

In [ ]:
import sympy
import mpmath

In [ ]:
sympy.init_printing()

# Simpson's rule

In [ ]:
a, b, X = sympy.symbols("a, b, x")
f = sympy.Function("f")

In [ ]:
#x = a, (a+b)/3, 2 * (a+b)/3, b # 3rd order quadrature rule
x = a, (a+b)/2, b # simpson's rule
#x = a, b # trapezoid rule
#x = ((b+a)/2,)  # mid-point rule

In [ ]:
w = [sympy.symbols("w_%d" % i) for i in range(len(x))] 

In [ ]:
q_rule = sum([w[i] * f(x[i]) for i in range(len(x))])

In [ ]:
q_rule

In [ ]:
phi = [sympy.Lambda(X, X**n) for n in range(len(x))]

In [ ]:
phi

In [ ]:
eqs = [q_rule.subs(f, phi[n]) - sympy.integrate(phi[n](X), (X, a, b)) for n in range(len(phi))]

In [ ]:
eqs

In [ ]:
w_sol = sympy.solve(eqs, w)

In [ ]:
w_sol

In [ ]:
q_rule.subs(w_sol).simplify()

## SciPy `integrate`

### Simple integration example

In [ ]:
def f(x):
    return np.exp(-x**2)

In [ ]:
val, err = integrate.quad(f, -1, 1)

In [ ]:
val

In [ ]:
err

In [ ]:
val, err = integrate.quadrature(f, -1, 1)

In [ ]:
val

In [ ]:
err

### Extra arguments

In [ ]:
def f(x, a, b, c):
    return a * np.exp(-((x-b)/c)**2)

In [ ]:
val, err = integrate.quad(f, -1, 1, args=(1, 2, 3))

In [ ]:
val

In [ ]:
err

### Reshuffle arguments

In [ ]:
from scipy.special import jv

In [ ]:
val, err = integrate.quad(lambda x: jv(0, x), 0, 5)

In [ ]:
val

In [ ]:
err

### Infinite limits 

In [ ]:
f = lambda x: np.exp(-x**2)

In [ ]:
val, err = integrate.quad(f, -np.inf, np.inf)

In [ ]:
val

In [ ]:
err

### Singularity

In [ ]:
f = lambda x: 1/np.sqrt(abs(x))

In [ ]:
a, b = -1, 1

In [ ]:
integrate.quad(f, a, b)

In [ ]:
integrate.quad(f, a, b, points=[0])

In [ ]:
fig, ax = plt.subplots(figsize=(8, 3))

x = np.linspace(a, b, 10000)
ax.plot(x, f(x), lw=2)
ax.fill_between(x, f(x), color='green', alpha=0.5)
ax.set_xlabel("$x$", fontsize=18)
ax.set_ylabel("$f(x)$", fontsize=18)
ax.set_ylim(0, 25)

fig.tight_layout()
fig.savefig("ch8-diverging-integrand.pdf")

## Tabulated integrand

In [ ]:
f = lambda x: np.sqrt(x)

In [ ]:
a, b = 0, 2

In [ ]:
x = np.linspace(a, b, 25)

In [ ]:
y = f(x)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 3))
ax.plot(x, y, 'bo')
xx = np.linspace(a, b, 500)
ax.plot(xx, f(xx), 'b-')
ax.fill_between(xx, f(xx), color='green', alpha=0.5)
ax.set_xlabel(r"$x$", fontsize=18)
ax.set_ylabel(r"$f(x)$", fontsize=18)
fig.tight_layout()
fig.savefig("ch8-tabulated-integrand.pdf")

In [ ]:
val_trapz = integrate.trapz(y, x)

In [ ]:
val_trapz

In [ ]:
val_simps = integrate.simps(y, x)

In [ ]:
val_simps

In [ ]:
val_exact = 2.0/3.0 * (b-a)**(3.0/2.0)

In [ ]:
val_exact

In [ ]:
val_exact - val_trapz

In [ ]:
val_exact - val_simps

In [ ]:
x = np.linspace(a, b, 1 + 2**6)

In [ ]:
len(x)

In [ ]:
y = f(x)

In [ ]:
val_exact - integrate.romb(y, dx=(x[1]-x[0]))

In [ ]:
val_exact - integrate.simps(y, dx=x[1]-x[0])

## Higher dimension

In [ ]:
def f(x):
    return np.exp(-x**2)

In [ ]:
%time integrate.quad(f, a, b)

In [ ]:
def f(x, y):
    return np.exp(-x**2-y**2)

In [ ]:
a, b = 0, 1

In [ ]:
g = lambda x: 0

In [ ]:
h = lambda x: 1

In [ ]:
integrate.dblquad(f, a, b, g, h)

In [ ]:
integrate.dblquad(lambda x, y: np.exp(-x**2-y**2), 0, 1, lambda x: 0, lambda x: 1)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))

x = y = np.linspace(-1.25, 1.25, 75)
X, Y = np.meshgrid(x, y)

c = ax.contour(X, Y, f(X, Y), 15, cmap=mpl.cm.RdBu, vmin=-1, vmax=1)

bound_rect = plt.Rectangle((0, 0), 1, 1,
                           facecolor="grey")
ax.add_patch(bound_rect)

ax.axis('tight')
ax.set_xlabel('$x$', fontsize=18)
ax.set_ylabel('$y$', fontsize=18)

fig.tight_layout()
fig.savefig("ch8-multi-dim-integrand.pdf")

In [ ]:
integrate.dblquad(f, 0, 1, lambda x: -1 + x, lambda x: 1 - x)

In [ ]:
def f(x, y, z):
    return np.exp(-x**2-y**2-z**2)

In [ ]:
integrate.tplquad(f, 0, 1, lambda x : 0, lambda x : 1, lambda x, y : 0, lambda x, y : 1)

In [ ]:
integrate.nquad(f, [(0, 1), (0, 1), (0, 1)])

### nquad

In [ ]:
def f(*args):
    return  np.exp(-np.sum(np.array(args)**2))

In [ ]:
%time integrate.nquad(f, [(0,1)] * 1)

In [ ]:
%time integrate.nquad(f, [(0,1)] * 2)

In [ ]:
%time integrate.nquad(f, [(0,1)] * 3)

In [ ]:
%time integrate.nquad(f, [(0,1)] * 4)

In [ ]:
%time integrate.nquad(f, [(0,1)] * 5)

### Monte Carlo integration

In [ ]:
from skmonaco import mcquad

In [ ]:
%time val, err = mcquad(f, xl=np.zeros(5), xu=np.ones(5), npoints=100000)

In [ ]:
val, err

In [ ]:
%time val, err = mcquad(f, xl=np.zeros(10), xu=np.ones(10), npoints=100000)

In [ ]:
val, err

## Symbolic and multi-precision quadrature

In [ ]:
x = sympy.symbols("x")

In [ ]:
f = 2 * sympy.sqrt(1-x**2)

In [ ]:
a, b = -1, 1

In [ ]:
sympy.plot(f, (x, -2, 2));

In [ ]:
val_sym = sympy.integrate(f, (x, a, b))

In [ ]:
val_sym

In [ ]:
mpmath.mp.dps = 75

In [ ]:
f_mpmath = sympy.lambdify(x, f, 'mpmath')

In [ ]:
val = mpmath.quad(f_mpmath, (a, b))

In [ ]:
sympy.sympify(val)

In [ ]:
sympy.N(val_sym, mpmath.mp.dps+1) - val

In [ ]:
%timeit mpmath.quad(f_mpmath, [a, b])

In [ ]:
f_numpy = sympy.lambdify(x, f, 'numpy')

In [ ]:
%timeit integrate.quad(f_numpy, a, b)

### double and triple integrals

In [ ]:
def f2(x, y):
    return np.cos(x)*np.cos(y)*np.exp(-x**2-y**2)

def f3(x, y, z):
    return np.cos(x)*np.cos(y)*np.cos(z)*np.exp(-x**2-y**2-z**2)

In [ ]:
integrate.dblquad(f2, 0, 1, lambda x : 0, lambda x : 1)

In [ ]:
integrate.tplquad(f3, 0, 1, lambda x : 0, lambda x : 1, lambda x, y : 0, lambda x, y : 1)

In [ ]:
x, y, z = sympy.symbols("x, y, z")

In [ ]:
f2 = sympy.cos(x)*sympy.cos(y)*sympy.exp(-x**2-y**2)

In [ ]:
f3 = sympy.cos(x)*sympy.cos(y)*sympy.cos(z) * sympy.exp(-x**2 - y**2 - z**2)

In [ ]:
sympy.integrate(f3, (x, 0, 1), (y, 0, 1), (z, 0, 1))  # this does not succeed

In [ ]:
f2_numpy = sympy.lambdify((x, y), f2, 'numpy')

In [ ]:
integrate.dblquad(f2_numpy, 0, 1, lambda x: 0, lambda x: 1)

In [ ]:
f3_numpy = sympy.lambdify((x, y, z), f3, 'numpy')

In [ ]:
integrate.tplquad(f3_numpy, 0, 1, lambda x: 0, lambda x: 1, lambda x, y: 0, lambda x, y: 1)

In [ ]:
mpmath.mp.dps = 30

In [ ]:
f2_mpmath = sympy.lambdify((x, y), f2, 'mpmath')

In [ ]:
res = mpmath.quad(f2_mpmath, (0, 1), (0, 1))
res

In [ ]:
f3_mpmath = sympy.lambdify((x, y, z), f3, 'mpmath')

In [ ]:
res = mpmath.quad(f3_mpmath, (0, 1), (0, 1), (0, 1))

In [ ]:
sympy.sympify(res)

## Line integrals

In [ ]:
t, x, y = sympy.symbols("t, x, y")

In [ ]:
C = sympy.Curve([sympy.cos(t), sympy.sin(t)], (t, 0, 2 * sympy.pi))

In [ ]:
sympy.line_integrate(1, C, [x, y])

In [ ]:
sympy.line_integrate(x**2 * y**2, C, [x, y])

## Integral transformations

### Laplace transforms

In [ ]:
s = sympy.symbols("s")

In [ ]:
a, t = sympy.symbols("a, t", positive=True)

In [ ]:
f = sympy.sin(a*t)

In [ ]:
sympy.laplace_transform(f, t, s)

In [ ]:
F = sympy.laplace_transform(f, t, s, noconds=True)

In [ ]:
F

In [ ]:
sympy.inverse_laplace_transform(F, s, t, noconds=True)

In [ ]:
[sympy.laplace_transform(f, t, s, noconds=True) for f in [t, t**2, t**3, t**4]]

In [ ]:
n = sympy.symbols("n", integer=True, positive=True)

In [ ]:
sympy.laplace_transform(t**n, t, s, noconds=True)

In [ ]:
sympy.laplace_transform((1 - a*t) * sympy.exp(-a*t), t, s, noconds=True)

### Fourier Transforms

In [ ]:
w = sympy.symbols("omega")

In [ ]:
f = sympy.exp(-a*t**2)

In [ ]:
F = sympy.fourier_transform(f, t, w)

In [ ]:
F

In [ ]:
sympy.inverse_fourier_transform(F, w, t)

In [ ]:
sympy.fourier_transform(sympy.cos(t), t, w)  # not good

## Versions

In [ ]:
%reload_ext version_information

In [ ]:
%version_information numpy, matplotlib, scipy, sympy, mpmath, skmonaco